In [ ]:
import pytest
from example_data import DummyTaskCanFail, example_data

from intelligence_layer.evaluation.run.in_memory_run_repository import (
    InMemoryRunRepository,
)
from intelligence_layer.evaluation.run.runner import Runner

my_example_data = example_data()

dataset_repository = my_example_data.dataset_repository
run_repository = InMemoryRunRepository()
task = DummyTaskCanFail()

runner = Runner(task, dataset_repository, run_repository, "MyRunDescription")

# How to resume a run after a crash

0. Run task on a dataset, see [here](./how_to_run_a_task_on_a_dataset.ipynb).
1. A crash occurs.
2. Re-run task on the same dataset with `resume_from_recovery_data` set to `True`.

In [ ]:
# Steps 0 & 1: Run task for dataset
with pytest.raises(Exception):  # noqa: B017
    run_overview = runner.run_dataset(my_example_data.dataset.id, abort_on_error=True)

A failure has occurred. Note, this might be a crash of the computer or an unexpected uncaught exception. 

For demonstration purposes, we set `abort_on_error=True`, such that an exception is raised. Further, we catch the exception for purely technical reasons of our CI. Feel free to remove the pytest scope on your local setup when running this notebook.

Even though the run crashed, the `RunRepository` stores recovery data and is able to continue `run_dataset` by setting `resume_from_recovery_data` to `True`. This way, the already successfully calculated outputs do not have to be re-calculated again, and only the missing examples are processed:

In [ ]:
# Step 2: Re-run the same run with `resume_from_recovery_data` enabled
run_overview = runner.run_dataset(
    my_example_data.dataset.id, abort_on_error=True, resume_from_recovery_data=True
)

In [ ]:
print(run_overview)

Note: The `FileSystemRepository` persists the recovery data in the file system. The run can therefore be resumed even in case of a complete program or even computer crash. 

On the other hand, the `InMemoryRunRepository` retains the recovery data only as long as the repository resides in computer memory. A crash of the process will lead to the loss of the recovery data. In that case, all examples will have to be recalculated.